# Introduction to Quantum Cryptography
In this notebook we are going to introduce the theory of **Quantum Cryptography** and some of its applications. 

The **preface** consists in a quick recap about traditional cryptography, its progress, and the reasons that led to the conception of quantum cryptography and its contemporary achievements. 

Following up, we recall some useful concepts of **quantum mechanics** without being too pedantic, more as a tool needed to argue the enforcement of this practice compared to classical crypto. In fact, this tutorial is intended to be divulgative and concise, not to substitute a scientific publication; to maintain the aim of it being more accessible, we renounce to some formality (studies will be linked where possible). 

## Requirements
Throughout the notebook we will make use of:
- Python 3.6 (although python 3.5 works as well)
- QISKit (with a Qconfig file for remote backend) 
- Standard python scientific libs stack: numpy etc.
- Knowledge of basic Quantum Mechanics is assumed
- Notions of traditional Cryptography are helpful although not strictly necessary

## Preface
Traditional **Cryptography** uses mathematical techniques to secure the communication of information. The procedures, that explain how to perform the operations, give shape to a certain **protocol** or **cipher**, a policy to be followed to enforce the achievement of different cryptographic tasks, such as: confidentiality, authenticity and data integrity. 

The related field of **cryptanalysis** focuses on protocol breaking, elaborating its weaknesses, and  finding a way of getting access to secret information. The cat-and-mouse game played by cryptologists and cryptoanalysts has pushed the field and driven progress in it tremendously. 

The reliability of a protocol is defined as the resilience to a certain attack performed in a given amount of time. Thus, it was thought that there was not such a thing as **intrinsic security** and that, given enough time or enough resource of computation any protocol would be broken. This is actually not true; perfectly concealing a message is indeed possible, although it is somewhat inconvenient to implement. 

Now suppose we have the simple scenario: a sender, **Alice**, who wants to deliver a secret message to a recipient, **Bob**, unbeknown of **Eve**, an eavesdropper. Alice takes the **plaintext**, the unencrypted information, and produces the **ciphertext** with a given criterion, using a **cryptographic key**. 

This key is a string of bits that is applied to the plaintext using a mathematical function, such as the bitwise XOR or a transposition/substitution map etc. The encrypted message is sent over a channel and delivered to Bob, who also possesses the same cryptographic key and performs the inverse operation, he applies the key to the ciphertext to recover the plaintext. 

In the case in which they have pre-shared the **same** key, called **symmetric cryptography**, this is the only element that Eve, who has intercepted only the ciphertext, needs to retrieve the plaintext message. Simple ciphers, such as transposition and substitution maps, are vulnerable to frequency analysis, meaning that the plaintext characters could be inferred counting the occurrences of characters in the ciphertext, and using statistical methods plaintext words can be reconstructed. Once the relations have been established, the key is obtained. 

To ensure perfect secrecy one would have to use a completely random generated key, long as much as the plaintext, and never use it again, a cipher known as **one-time pad** (OTP). Once this random key is generated, Alice can apply for example the bitwise XOR to the plaintext and, being the XOR symmetrical, Bob just needs to reapply the XOR with the same key to the ciphertext to obtain the plaintext. 

I stress the word random because this is the whole strength of the protocol: if Eve attempts to bruteforce the ciphertext and tries all the different keys to a one-word message for example, she would eventually find the right word, but she would find as well all the other possible one-words and she would be unable to decide which one to pick! 

However OTP has never gained widespread popularity for several practical reasons: there is not such a thing as a perfectly random key, careful storing of the key (and safe disposal to prevent any reuse) is a problem, and going through the generation of a pre-shared key for every message to send is a huge effort. The latter is the biggest problem of all and it makes this cipher extremely impractical for real use cases.

The invention of **asymmetric cryptography**, also known as **public-key cryptography**, has been the most important achievement of modern cryptography. In this system each user generates a pair of keys, a **public key**, spread freely to everyone, and a **private key**, kept in maximum secrecy. Alice can encrypt a message using Bob's public key, but only Bob is able to decrypt it using his private key. 

However, to implement this feature, we have to use mathematical functions that are generally difficult to solve but, once a solution is known, it is not so difficult to verify it. Contemporary implementations of mathematical algorithms make use of integer factorization, discrete logarithm, and elliptic curve relationships. It is not the scope of this notebook to go into the details of this process, which is widely discussed in literature. 

Let's just say it offers a good compromise between security and practicality, overcoming the problem of one-time key generation and, most importantly, **key exchange**. In fact in real use cases almost all major implementations of cryptography are realized asymmetrically (although sometimes in combination with symmetrical keys), such as TLS, S/MIME, PGP, OMEMO, OTR etc.

The advent of **quantum computers** has raised an unexpected problem for public key cryptography. It has been proven that a quantum computer is able to solve all mathematical algorithms currently in use in a much more efficient way. Thanks to the properties of the **Quantum Fourier Transform** (QFT) and its implementation in Shor's algorithm, the time needed to find a general solution for the problem is reduced from being exponential to  being only polynomial, thus opening a window for possible brute-force attacks. 

At the present time quantum computers are not powerful enough to actually perform these operations on big numbers, but in the not so distant future they might as well be, so much so that cryptography experts have begun to discuss strategies to harden cryptographic protocols in case of widely available quantum computers; this field is called **post-quantum cryptography**.  

Quantum computers also present another extraordinary advantage, that may lead to perfect secrecy. Storing information in the form of qubits is the first step in implementing **quantum cryptography**, the science of safely exchanging information using quantum physics. This is achieved through the establishment of a **quantum channel**, a special apparatus capable of delivering quantum information in the form of qubits. Confidentiality is guaranteed by laws of quantum mechanics, a remarkable way of calling physics in aid of mathematics. 

We will see in the next notebook how **quantum key distribution** solves, once and for all (?), the problems associated with key sniffing. Although it may seem hard to do in practice, it has actually been accomplished multiple times, given the right machinery, up to the realization of quantum encrypted satellite communication. In the following section we recall relevant properties of quantum mechanics needed to put in place a quantum channel. 

## Quantum mechanics
### Qubits
A **quantum channel** provides a medium and an apparatus to exchange quantum information between two parties using **qubits**, a set of two state quantum mechanical systems. There are many kinds of qubits such as electrons, photons, quantum dots, Josephson junctions etc. **Polarized photons** were the first type of qubits to be used in practice and they continue to be popular today, since they are relatively easy to obtain compared to other qubits, and they can be exchanged either through fiber optic cables or via laser beacons. According to quantum mechanics, electromagnetic waves can be thought as stream of photons. The term **polarization** classically refers to the direction of the electric field of the wave, but can be also seen as an intrinsic  property of single photons. Natural sources of light are incoherent and unpolarized because they consist of a random mixture of waves having different frequencies, phases, and polarization states, but polarized light can be produced by passing light through a **polarizing filter**, which allows waves of only one polarization to pass through.

Consider a **monochromatic electromagnetic wave** that propagates along the $\hat{z}$ axis. The electric field, $\vec{E}$, lies in the $\hat{x}\hat{y}$ plane, the angle $\theta$ that it makes with the $\hat{x}$ axis is the  angle of monochromatic radiation. In general the evolution of the electric field in time can be described according to the following equation (using complex notation):

\begin{equation}
   \vec{E}(\vec{r},t)=|\vec{E}|\ \Re\big{(}\lvert\psi\rangle e^{i(kz - \omega t)}\ \big{)}
\end{equation}

where $|\vec{E}|$ is the amplitude of the electric field, $\omega$ is the angular frequency of the wave in terms of the wave number $k$, and $ \lvert\psi\rangle = \big{(} cos\theta e^{i\phi_x},\ sin\theta e^{i\phi_y}\big{)}$ is **Jones vector** that describes the **polarization** of the photon. This vector is composed of two complex amplitudes, each with its own magnitude and phase, and represents the projection of $\vec{E}$ on the $\hat{x}$ and $\hat{y}$ axis, so that $|E_x| = |\vec{E}|cos\theta$ and $|E_y| = |\vec{E}|sin\theta$, and thus $|\vec{E}|^2 = |E_x|^2 + |E_y|^2$. These phases are usually absorbed in an overall phase $\phi=\phi_x - \phi_y$ because, as $t$ and $z$ change, $\phi_x$ and $\phi_y$ change in the same way so that $\phi$ is actually unchanged in the evolution. The complex notation can be somewhat scary to non accustomed readers, but it's easy to rewrite the general expression in a real form:

\begin{equation}
   \vec{E}(\vec{r},t)=E_x\ cos(kz-\omega t+\phi_x)\ \hat{x}+E_y\ cos(kz-\omega t+\phi_y)\ \hat{y}
\end{equation}

The wave is said to be **linearly polarized** if $\phi = 0$, there is no phase difference between $E_x$ and $E_y$ and $\phi_x=\phi_y$. If we set $z=0$ and fix the phases at $\phi_x=\phi_y=0$ we obtain a very simple representation of linear polarization in a generic direction, at an angle $\theta$ with respect to the $\hat{x}$ axis:

\begin{equation}
   \vec{E}(\vec{r},t)=E_x\ cos(\omega t)\ \hat{x}+E_y\ cos(\omega t)\ \hat{y} = |\vec{E}|\ cos\theta\ cos(\omega t)\ \hat{x}+|\vec{E}|\ sin\theta\ cos(\omega t)\ \hat{y}
\end{equation}

We can see explicitly that the electric field oscillates in time but maintains its direction constant.
If we have $sin\theta=0$, only the $\hat{x}$ component is present, and the wave is said to be linearly polarized in the $\hat{x}$ direction, typically called **horizontal polarization**; viceversa if $cos\theta=0$, only the $\hat{y}$ component is present, and the wave is said to be linearly polarized in the $\hat{y}$ direction, typically called **vertical polarization**.

This prompts us to introduce orthonormal unit vectors such that:

\begin{equation}
\lvert 0 \rangle =
\lvert x \rangle =
  \begin{pmatrix}
    1 \\
    0 \\
    \end{pmatrix}
    , \ \ \  
    \lvert 1 \rangle =
    \lvert y \rangle = 
  \begin{pmatrix}
    0 \\
    1 \\
    \end{pmatrix}
\end{equation}

so that the generic linearly polarized state can be written, in terms of the Jones vector, as:

\begin{equation}
\lvert \psi \rangle = e^{i \alpha}\big{(}cos\theta \lvert 0 \rangle + sin\theta \lvert 1 \rangle\big{)}
\end{equation}

If it looks familiar it is probably because the $\lvert 0 \rangle$ and $\lvert 1 \rangle$ vectors are the usual qubits used in the [Bloch sphere](https://www.qiskit.org/ibmqx-user-guides/beginners-guide/004-The_Weird_and_Wonderful_World_of_the_Qubit/001-The_Weird_and_Wonderful_World_of_the_Qubit.html). But these are not the only states that we can use with polarized photons. It's useful to introduce the quantum superpositions of these two states, that together make up another valid orthonormal basis called **Hadamard basis** that we will use, and the vectors of the photons that represent **diagonal polarization**, or linearly polarized at 45° from the $\hat{x}$ axis, and **anti-diagonal polarization**, or linearly polarized at −45° from the $\hat{x}$ axis:

\begin{equation}
\lvert \nearrow \rangle =
\frac{1}{\sqrt{2}} \big{(}\lvert 0 \rangle + \lvert 1 \rangle \big{)}= 
  \frac{1}{\sqrt{2}}
  \begin{pmatrix}
    1 \\
    1 \\
    \end{pmatrix}
    , \ \ \ \ 
\lvert \searrow \rangle =
\frac{1}{\sqrt{2}} \big{(}\lvert 0 \rangle - \lvert 1 \rangle \big{)}= 
  \frac{1}{\sqrt{2}}
  \begin{pmatrix}
    1 \\
    -1 \\
    \end{pmatrix}    
\end{equation}

From the equation of the electric field it's straightforward to see that in linear polarization, the fields oscillate in a single direction. Also note that diagonal polarization states, are linear combinations of horizontal and vertical polarization states, and thus they are **not orthogonal** to these. For the sake of completeness, we have to mention that the electric field can also rotate at a constant rate with respect to $\hat{z}$ in which case we would have **circular polarization**. The rotation can have two possible directions: clockwise or anti-clockwise anti-clockwise: if the fields rotate clockwise, the result is called right circular polarization, viceversa if they rotate anti-clockwise, it is called left circular polarization:

\begin{equation}
\lvert \circlearrowright \rangle =
\frac{1}{\sqrt{2}} \big{(}\lvert 0 \rangle - i \lvert 1 \rangle \big{)}= 
  \frac{1}{\sqrt{2}}
  \begin{pmatrix}
    1 \\
    -i \\
    \end{pmatrix}
    , \ \ \ \ 
\lvert \circlearrowleft \rangle =
\frac{1}{\sqrt{2}} \big{(}\lvert 0 \rangle + i \lvert 1 \rangle \big{)}= 
  \frac{1}{\sqrt{2}}
  \begin{pmatrix}
    1 \\
    i \\
    \end{pmatrix}    
\end{equation}

Note that all the other points on the Bloch sphere that are not represented by these states are still valid polarization states for the photon, known as **elliptical polarization**, we will not make use of these states.  

Now that we know how to obtain and manipulate working qubits we want to know their characteristics that make them suitable for key sharing. In particular the two fundamental properties that help us in our task are the fact that a measurement of an unknown quantum state causes it to collapse in an unknown eigenstate (Heisenberg's principle) and that it's impossible to create an identical copy of an arbitrary quantum state (no-cloning theorem). 

### No-cloning Theorem
In traditional electronics, digital signals are carried by conductive wires or traces in which current flows. If we wish to clone an arbitrary unknown signal on another track, the only thing we need to do is to bifurcate the wire and, given the right accuracy on the circuitry, we end up with the same signal duplicated on two tracks. In quantum computers things work differently: if the state is in an arbitrary state, we have no way of making exact copies of it. This is due do the fact that, when we work with quantum systems, there are only two permissible quantum operations with which we may manipulate the state: we can perform an observation, which irreversibly collapses the system into some eigenstate of an observable, corrupting the information contained in the original state, or we can utilize a unitary operator.

Suppose that we have a quantum state $\lvert \psi \rangle$ that we wish to duplicate using a unitary operator, $U$: $\lvert \psi \rangle \rightarrow \lvert \psi \rangle\lvert \psi \rangle$. Since unitary transformations connect spaces of the same dimension we have to introduce a service state, $\lvert s \rangle$, so that:  

\begin{equation}
U\big{(}\lvert \psi \rangle \lvert s \rangle \big{)} = \lvert \psi \rangle\lvert \psi \rangle
\end{equation}

Select an arbitrary state, $\lvert \chi \rangle$. Unitary operators are useful because they conserve the scalar product between states. Recall the definition of a unitary operator: $U^{+}U = UU^{+}=I$, from which conservation of scalar product follows:

\begin{equation}
    \langle \psi \lvert \chi \rangle = \langle U\psi \lvert U\chi \rangle = \langle U^{+}U\psi \lvert \chi \rangle  = \langle \psi \lvert \chi \rangle
\end{equation}

Now, knowing that the cloning machine has to work for any arbitrary state, we can apply it to $\chi$ as well: $U\big{(}\lvert \chi \rangle \lvert s \rangle \big{)} = \lvert \chi \rangle\lvert \chi \rangle$. We take the scalar product between the two states and we apply $U$:

\begin{equation}
    \langle \psi \lvert \chi \rangle \langle s \lvert s \rangle = \langle \psi \lvert \chi \rangle \langle \psi \lvert \chi \rangle 
\end{equation}

We can normalize $\lvert s \rangle $ so that we obtain in the end:

\begin{equation}
    \langle \psi \lvert \chi \rangle = {\langle \psi \lvert \chi \rangle}^{2}
\end{equation}

This equation has only two solutions: $ \langle \psi \lvert \chi \rangle = 1$ or $ \langle \psi \lvert \chi \rangle = 0$ which means that either the two states are orthogonal or they are the same state, thus proving that we cannot perfectly clone an arbitrary state.

The no-cloning theorem has profound implications in quantum computing. It prevents classical **error correction** on quantum states, since backup copies of a state cannot be made in the middle of a quantum computation and used for correcting subsequent errors. It is also a requirement for the no-communication theorem, which states that it's impossible to trasmit quantum information over a quantum channel only by measuring entangled states, as outlined in the famous EPR experiment. 

The most relevant consequence of the no-cloning theorem for our case is the fact that it's impossible to distinguish non-orthogonal states with only one measurement. Suppose that we have a state $\lvert \psi \rangle$ that can be one of two states: either $\lvert 0 \rangle$ or $\lvert \nearrow \rangle = \frac{1}{\sqrt{2}} \big{(}\lvert 0 \rangle + \lvert 1 \rangle \big{)}$. We have at our service a measuring device that operates on the computational basis $\{\lvert 0 \rangle,\  \lvert 1 \rangle\}$. According to the principle of quantum mechanics, the result of the measure on the system will return one of the possible eigenvalues, either '$0$' or '$1$' and the state would collapse in the relative eigenstate. If we obtain '$1$' the state would now become $\lvert 1 \rangle$ and right away we know that we measured state $\lvert \nearrow \rangle$. But if we obtain '$0$' the state would collapse in $\lvert 0 \rangle$ and we have no way of saying in which one of the two states it was in before the measurement. If we had a cloning machine we could use it to make many copies of the state and, by performing a measurement on each state, we could use statistics to infer which state we began with. 

This property of non-orthogonal states is the core idea behind quantum cryptography. It's worth to introduce the first thought application of qubits in the real world that later served as an inspiration for quantum key sharing. Quantum money was the first conceptually idealized currency that used quantum mechanics to guarantee its security.  

## Quantum Money
In 1970 Stephen Wiesner, a graduate student at Columbia University, came up with an interesting idea that was actually rejected by several journals but was eventually published in 1983. He envisioned a new kind of bank note that, in addition to a unique serial number, would also contain a certain number of qubits. These qubits could be made using polarized photons, quantum dots, isolate nuclei or any other two state systems, the important thing is that they are fixed in a very specific state that only the bank knows. In particular each qubit is randomly set in an eigenstate of one of two basis that are not orthogonal to each other, such as $\{\lvert 0 \rangle,\  \lvert 1 \rangle\}$ and $\{\lvert \nearrow \rangle,\  \lvert \searrow \rangle\}$, and they are made in such a way that phase coherence is maintained in time.

The bank keeps a record composed of every bank note number associated with its quantum state so that, when questioned about the validity of the bank note, the bank performs a check to see if each qubit is still in its initial state or it has switched to the orthogonal state. Now consider a counterfeiter that tries to forge a new bank note from an old one. Only the bank knows the right sequence of basis used to measure the quantum state, so the counterfeiter can only guess which one of the four eigenstates each qubit is in but he needs to measure each qubit in one of the two bases to find its eigenstate. He has $1/2$ chance of picking the wrong basis, so he will guess rightly about half of the times, but even in the case he picks the wrong basis, in this event he has a probability $1/2$ of ending up in the right eigenstate, since the wrong basis can be seen as a superposition of the base elements of the right basis; thus, he will correctly guess each qubit with a probability of $3/4$. If we imagine to have $n$ qubits embedded on the bank note, the probability of getting all of them right and perfectly forge an identical bank note would be ${(3/4)}^{n}$; for n=20 this is  $0.00317$ or about $0.3\%$, which is low enough to discourage any potential forgery.

The act of observing the qubit, which is indeed a projective measure, causes the qubit to collapse in one of its eigenstate, so that not only the bank note is unsuccessfully cloned, but also the original bank note is now useless. This peculiar property of quantum mechanics, wave function collapse, in conjunction with non-orthogonal states, is what enforces security in quantum cryptography. 